# Error analysis

In [86]:
#dependencies
import pandas as pd
from label_legends.preprocess import load_test, load_data, transform, load_train
from label_legends.result import load_predictions, get_current, load_predictions
import json
import os

In [87]:
tra = load_train().collect()
train = tra.to_pandas()
train

,id,rewire_id,text,annotator,label_sexist,label_category,label_vector,split,tokens
0,0,sexism2022_english-0,I wonder what keeps that witch looking quite ...,17,sexist,2. derogation,2.2 aggressive and emotive attacks,train,"[i, wonder, what, keep, that, witch, look, qui..."
1,1,sexism2022_english-0,I wonder what keeps that witch looking quite ...,2,sexist,2. derogation,2.2 aggressive and emotive attacks,train,"[i, wonder, what, keep, that, witch, look, qui..."
2,10,sexism2022_english-100,"Good for her! My grandson had acid reflux, but...",3,not sexist,none,none,train,"[good, for, her, !, my, grandson, have, acid, ..."
3,1000,sexism2022_english-10297,The article said Madonna offered to give anyon...,5,sexist,2. derogation,2.3 dehumanising attacks & overt sexual object...,train,"[the, article, say, madonna, offer, to, give, ..."
4,10000,sexism2022_english-12998,Gasoline. The answer 60% of America gives when...,8,not sexist,none,none,train,"[gasoline, ., the, answer, 60, %, of, america,..."
...,...,...,...,...,...,...,...,...,...
41995,9992,sexism2022_english-12995,"haha woman suck amirite. it's like, obviously ...",1,sexist,2. derogation,2.1 descriptive attacks,train,"[haha, woman, suck, amirite, ., it, be, like, ..."
41996,9996,sexism2022_english-12997,You mean one that forces women to like wimpy f...,6,not sexist,none,none,train,"[you, mean, one, that, force, woman, to, like,..."
41997,9997,sexism2022_english-12997,You mean one that forces women to like wimpy f...,4,not sexist,none,none,train,"[you, mean, one, that, force, woman, to, like,..."
41998,9998,sexism2022_english-12997,You mean one that forces women to like wimpy f...,2,sexist,3. animosity,3.2 immutable gender differences and gender st...,train,"[you, mean, one, that, force, woman, to, like,..."


In [88]:
original = load_test().collect()
original = original.drop(["rewire_id", "label_category","label_vector","annotator"])
og_test = transform(original)
og_test

id,text,tokens,token_ids,label
i64,str,list[str],list[i64],i64
100,"""It is not insulting, it's **ex…","[""insulting"", "","", … "".**""]","[1407, 22, … 3000]",1
10005,"""and if you buy into the Christ…","[""buy"", ""christianity"", … ""..""]","[464, 549, … 27]",0
10006,"""and if you buy into the Christ…","[""buy"", ""christianity"", … ""..""]","[464, 549, … 27]",0
10007,"""and if you buy into the Christ…","[""buy"", ""christianity"", … ""..""]","[464, 549, … 27]",0
10008,"""Given the sub this is posted i…","[""sub"", ""post"", … ""shit""]","[2591, 2042, … 2399]",0
…,…,…,…,…
9985,"""Today in leftist violence, Dan…","[""today"", ""leftist"", … "".""]","[2714, 1553, … 26]",0
9986,"""Today in leftist violence, Dan…","[""today"", ""leftist"", … "".""]","[2714, 1553, … 26]",0
9993,"""Shudder.. if you had to have s…","[""shudder"", "".."", … "".""]","[3000, 27, … 26]",1


In [89]:
min(og_test["id"])
max(og_test["id"])

59996

In [90]:
main_folder = r"../resource/mlflow"

final_df = None

for dir in os.listdir(main_folder):
    path = os.path.join(main_folder, dir)

    if os.path.isdir(path):
        #get the final path to the json file
        json_file_path = os.path.join(path, "predictions.json")

        #loading the json file
        if os.path.exists(json_file_path):
            with open(json_file_path, "r") as file:
                data = json.load(file)

            df = pd.DataFrame(data["data"], columns=["id", f"model_{dir}_pred"])

            #merge the dataframes
            if final_df is None:
                final_df = df
            else:
                final_df = pd.merge(final_df, df, on="id", how="outer")

og_test = og_test.to_pandas()
final_df = pd.merge(og_test, final_df, on="id", how="outer")
final_df


,id,text,tokens,token_ids,label,model_DeBERTa_pred,model_distilbert_pred,model_mostfrequent_pred,model_Regex_pred,model_xgboost_pred
0,27,Happy Birthday to your daughter GTR😊,"[happy, birthday, daughter, gtr, 😊]","[1238, 363, 747, 3000, 2996]",0,0,0,0,0,0
1,28,Happy Birthday to your daughter GTR😊,"[happy, birthday, daughter, gtr, 😊]","[1238, 363, 747, 3000, 2996]",0,1,0,0,0,0
2,29,Happy Birthday to your daughter GTR😊,"[happy, birthday, daughter, gtr, 😊]","[1238, 363, 747, 3000, 2996]",0,1,0,0,0,0
3,30,NEGRO Baltimore mayor and NEGRO police commiss...,"[negro, baltimore, mayor, negro, police, commi...","[1823, 3000, 1680, 1823, 2022, 3000, 3000, 182...",0,0,0,0,1,0
4,31,NEGRO Baltimore mayor and NEGRO police commiss...,"[negro, baltimore, mayor, negro, police, commi...","[1823, 3000, 1680, 1823, 2022, 3000, 3000, 182...",0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
11995,59992,"You can stop right there asshole, she is a mar...","[stop, right, asshole, ,, married, woman, fami...","[2567, 2274, 257, 22, 1661, 2937, 1020, 26]",0,0,0,0,1,0
11996,59993,"You can stop right there asshole, she is a mar...","[stop, right, asshole, ,, married, woman, fami...","[2567, 2274, 257, 22, 1661, 2937, 1020, 26]",0,0,0,0,1,0
11997,59994,"Since 1973, America has sanctioned the murder ...","[1973, ,, america, sanction, murder, 58, milli...","[3000, 22, 195, 3000, 1782, 3000, 1722, 536, 1...",1,0,0,0,1,0
11998,59995,"Since 1973, America has sanctioned the murder ...","[1973, ,, america, sanction, murder, 58, milli...","[3000, 22, 195, 3000, 1782, 3000, 1722, 536, 1...",1,0,0,0,1,0


In [91]:
final_df["error_level"] = 5

for i, row in final_df.iterrows():
    true_label = row["label"]
    true_label_count = 0
    for column in final_df.columns:
        if "model" in column:
            pred_label = row[column]
            if pred_label == true_label:
                true_label_count += 1
    final_df.at[i,"error_level"] = 5 - true_label_count

final_df

,id,text,tokens,token_ids,label,model_DeBERTa_pred,model_distilbert_pred,model_mostfrequent_pred,model_Regex_pred,model_xgboost_pred,error_level
0,27,Happy Birthday to your daughter GTR😊,"[happy, birthday, daughter, gtr, 😊]","[1238, 363, 747, 3000, 2996]",0,0,0,0,0,0,0
1,28,Happy Birthday to your daughter GTR😊,"[happy, birthday, daughter, gtr, 😊]","[1238, 363, 747, 3000, 2996]",0,1,0,0,0,0,1
2,29,Happy Birthday to your daughter GTR😊,"[happy, birthday, daughter, gtr, 😊]","[1238, 363, 747, 3000, 2996]",0,1,0,0,0,0,1
3,30,NEGRO Baltimore mayor and NEGRO police commiss...,"[negro, baltimore, mayor, negro, police, commi...","[1823, 3000, 1680, 1823, 2022, 3000, 3000, 182...",0,0,0,0,1,0,1
4,31,NEGRO Baltimore mayor and NEGRO police commiss...,"[negro, baltimore, mayor, negro, police, commi...","[1823, 3000, 1680, 1823, 2022, 3000, 3000, 182...",0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
11995,59992,"You can stop right there asshole, she is a mar...","[stop, right, asshole, ,, married, woman, fami...","[2567, 2274, 257, 22, 1661, 2937, 1020, 26]",0,0,0,0,1,0,1
11996,59993,"You can stop right there asshole, she is a mar...","[stop, right, asshole, ,, married, woman, fami...","[2567, 2274, 257, 22, 1661, 2937, 1020, 26]",0,0,0,0,1,0,1
11997,59994,"Since 1973, America has sanctioned the murder ...","[1973, ,, america, sanction, murder, 58, milli...","[3000, 22, 195, 3000, 1782, 3000, 1722, 536, 1...",1,0,0,0,1,0,4
11998,59995,"Since 1973, America has sanctioned the murder ...","[1973, ,, america, sanction, murder, 58, milli...","[3000, 22, 195, 3000, 1782, 3000, 1722, 536, 1...",1,0,0,0,1,0,4


In [92]:
final_df = final_df.sort_values(by="error_level", ascending=False)
error_df = final_df[final_df['error_level'] == 5]
display(error_df)

,id,text,tokens,token_ids,label,model_DeBERTa_pred,model_distilbert_pred,model_mostfrequent_pred,model_Regex_pred,model_xgboost_pred,error_level
11958,59787,"Don’t misunderstand me, I love my mother to de...","[misunderstand, ,, love, mother, death, ., tim...","[3000, 22, 1616, 1764, 755, 26, 2707, 22, 1579...",1,0,0,0,0,0,5
4423,22021,she forgot she was a mom....can we burn her too?,"[forget, mom, ...., burn, ?]","[1097, 1750, 29, 454, 98]",1,0,0,0,0,0,5
4424,22022,she forgot she was a mom....can we burn her too?,"[forget, mom, ...., burn, ?]","[1097, 1750, 29, 454, 98]",1,0,0,0,0,0,5
9968,49940,"Yeah mate , in my 3+ years working in IT i hav...","[yeah, mate, ,, 3, +, year, work, learn, old, ...","[2966, 1672, 22, 66, 21, 2967, 2944, 1549, 188...",1,0,0,0,0,0,5
11289,56424,F - FACE: Has it fallen on one side? A - ARMS:...,"[f, -, face, :, fall, ?, -, arms, :, raise, ?,...","[1001, 23, 1002, 86, 1015, 98, 23, 3000, 86, 2...",1,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...
2311,11518,I don't care if she doesn't care. If she tries...,"[care, care, ., try, break, house, shoot, ., d...","[487, 487, 26, 2779, 418, 1317, 2407, 26, 751,...",1,0,0,0,0,0,5
2312,11519,I don't care if she doesn't care. If she tries...,"[care, care, ., try, break, house, shoot, ., d...","[487, 487, 26, 2779, 418, 1317, 2407, 26, 751,...",1,0,0,0,0,0,5
11975,59894,The next person to play James Bond will be fem...,"[person, play, james, bond, female, ., watch, ...","[1973, 2012, 1450, 387, 1047, 26, 2887, 1235, 26]",1,0,0,0,0,0,5
11982,59949,NOT MY PROBLEM LOL I doubt it is as bad as rep...,"[problem, lol, doubt, bad, reput, ..., know, i...","[2090, 1602, 859, 297, 3000, 28, 1511, 1434, 2...",1,0,0,0,0,0,5


In [93]:
from collections import Counter

def count_tokens(tokens):
    return dict(Counter(tokens))

df = pd.DataFrame()
df["id"]= final_df["id"]
df["text"]= final_df["text"]
df["error_level"] = final_df["error_level"]
df['token_counts'] = final_df['tokens'].apply(count_tokens)
df

,id,text,error_level,token_counts
11958,59787,"Don’t misunderstand me, I love my mother to de...",5,"{'misunderstand': 1, ',': 2, 'love': 1, 'mothe..."
4423,22021,she forgot she was a mom....can we burn her too?,5,"{'forget': 1, 'mom': 1, '....': 1, 'burn': 1, ..."
4424,22022,she forgot she was a mom....can we burn her too?,5,"{'forget': 1, 'mom': 1, '....': 1, 'burn': 1, ..."
9968,49940,"Yeah mate , in my 3+ years working in IT i hav...",5,"{'yeah': 1, 'mate': 1, ',': 1, '3': 1, '+': 1,..."
11289,56424,F - FACE: Has it fallen on one side? A - ARMS:...,5,"{'f': 1, '-': 4, 'face': 1, ':': 3, 'fall': 1,..."
...,...,...,...,...
8260,41251,CliffyB is a classic case of more money than b...,0,"{'cliffyb': 1, 'classic': 1, 'case': 1, 'money..."
8259,41250,CliffyB is a classic case of more money than b...,0,"{'cliffyb': 1, 'classic': 1, 'case': 1, 'money..."
8223,41028,Cut it in her own flesh while looking in a mir...,0,"{'cut': 1, 'flesh': 1, 'look': 1, 'mirror': 1,..."
8235,41130,"The Iron Maiden, the toughest tranny in town.",0,"{'iron': 1, 'maiden': 1, ',': 1, 'tough': 1, '..."


## Are all test tokens in train tokens?

In [94]:
train_tokens = set(token for tokens in train["tokens"] for token in tokens)

# Flatten and get unique tokens from the test dataset
test_tokens = set(token for tokens in final_df["tokens"] for token in tokens)

error_tokens = set(token for tokens in error_df["tokens"] for token in tokens)

# Find tokens in the test dataset that are not in the train dataset
missing_tokens = test_tokens - train_tokens
len(missing_tokens)

error_in_missing = error_tokens.intersection(missing_tokens)
error_token_rate = len(error_in_missing) / len(error_tokens) if len(error_tokens) > 0 else 0
error_token_rate

0.14087061668681983

In [98]:
from collections import Counter

#list of all the token in error_df
all_tokens = [token for tokens in error_df['tokens'] for token in tokens]

#count each token in the list
token_counts = Counter(all_tokens)

#sort the dataframe
token_count_df = pd.DataFrame(token_counts.items(), columns=['token', 'count'])
token_count_df = token_count_df.sort_values(by='count', ascending=False)

print(token_count_df.head(20))

      token  count
5         .    658
1         ,    382
126   woman    227
15        ?     86
68      man     79
54     girl     78
7      like     75
85        "     67
79   female     54
38        -     50
33     just     48
67      say     46
35     good     45
48      ...     45
173    know     36
472    look     35
381    want     33
304       #     33
158    make     31
281   think     31


In [99]:
high_error_df = final_df[final_df['error_level'] > 3]
display(high_error_df)

,id,text,tokens,token_ids,label,model_DeBERTa_pred,model_distilbert_pred,model_mostfrequent_pred,model_Regex_pred,model_xgboost_pred,error_level
11958,59787,"Don’t misunderstand me, I love my mother to de...","[misunderstand, ,, love, mother, death, ., tim...","[3000, 22, 1616, 1764, 755, 26, 2707, 22, 1579...",1,0,0,0,0,0,5
4423,22021,she forgot she was a mom....can we burn her too?,"[forget, mom, ...., burn, ?]","[1097, 1750, 29, 454, 98]",1,0,0,0,0,0,5
4424,22022,she forgot she was a mom....can we burn her too?,"[forget, mom, ...., burn, ?]","[1097, 1750, 29, 454, 98]",1,0,0,0,0,0,5
9968,49940,"Yeah mate , in my 3+ years working in IT i hav...","[yeah, mate, ,, 3, +, year, work, learn, old, ...","[2966, 1672, 22, 66, 21, 2967, 2944, 1549, 188...",1,0,0,0,0,0,5
11289,56424,F - FACE: Has it fallen on one side? A - ARMS:...,"[f, -, face, :, fall, ?, -, arms, :, raise, ?,...","[1001, 23, 1002, 86, 1015, 98, 23, 3000, 86, 2...",1,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...
2258,11168,"Ironically, men are actually getting paid prop...","[ironically, ,, man, actually, pay, proportion...","[3000, 22, 1651, 143, 1949, 3000, 22, 2013, 83...",1,0,1,0,0,0,4
11952,59766,Yay! Nothing like a poor imitation of a women ...,"[yay, !, like, poor, imitation, woman, 4, inch...","[3000, 0, 1579, 2029, 3000, 2937, 71, 1373, 807]",1,0,0,0,1,0,4
11998,59995,"Since 1973, America has sanctioned the murder ...","[1973, ,, america, sanction, murder, 58, milli...","[3000, 22, 195, 3000, 1782, 3000, 1722, 536, 1...",1,0,0,0,1,0,4
11997,59994,"Since 1973, America has sanctioned the murder ...","[1973, ,, america, sanction, murder, 58, milli...","[3000, 22, 195, 3000, 1782, 3000, 1722, 536, 1...",1,0,0,0,1,0,4


In [100]:
high_error_tokens = set(token for tokens in high_error_df["tokens"] for token in tokens)

# Find tokens in the test dataset that are not in the train dataset
missing_tokens = test_tokens - train_tokens
len(missing_tokens)

high_error_in_missing = high_error_tokens.intersection(missing_tokens)
high_error_token_rate = len(high_error_in_missing) / len(high_error_tokens) if len(high_error_tokens) > 0 else 0
high_error_token_rate

0.16982266305981364

In [101]:
all_tokens_high = [token for tokens in high_error_df['tokens'] for token in tokens]

#count each token in the list
high_token_counts = Counter(all_tokens_high)

#sort the dataframe
high_token_count_df = pd.DataFrame(high_token_counts.items(), columns=['token', 'count'])
high_token_count_df = high_token_count_df.sort_values(by='count', ascending=False)

print(high_token_count_df.head(20))

       token  count
5          .   2055
1          ,   1064
126    woman    738
85         "    270
15         ?    212
7       like    211
68       man    209
54      girl    166
33      just    155
79    female    149
38         -    127
35      good    120
67       say    119
381     want    116
177        !    114
48       ...    114
173     know    104
1663    fuck     95
472     look     93
158     make     87
